In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import numpy as np

Load Dataset:<br>
train_dataset=original training set with no augmentation<br>
back_dataset=training set with back-translation<br>
aug_dataset=training set with eda-paraphrasing augmentation

In [2]:
train_dataset=pd.read_csv(r'..\DAIC\Preprocessed\train_dataset.csv')
back_dataset=pd.read_csv(r'..\DAIC\Preprocessed\back_dataset.csv')
aug_dataset=pd.read_csv(r'..\DAIC\Preprocessed\aug_dataset.csv')
test_dataset=pd.read_csv(r'..\DAIC\Preprocessed\test_dataset.csv')
val_dataset=pd.read_csv(r'..\DAIC\Preprocessed\dev_dataset.csv')

Checking the no of datapoints and class balances.

In [3]:
print(f'The samples in training dataset is: ',(len(train_dataset['response'])),'and the distribution is ',(train_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in back dataset is: ',(len(back_dataset['response'])),'and the distribution is ',(back_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in aug dataset is: ',(len(aug_dataset['response'])),'and the distribution is ',(aug_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in validation dataset is: ',(len(val_dataset['response'])),'and the distribution is ',(val_dataset['PHQ8_Binary'].value_counts()))
print(f'The samples in test dataset is: ',(len(test_dataset['response'])),'and the distribution is ',(test_dataset['PHQ8_Binary'].value_counts()))

The samples in training dataset is:  107 and the distribution is  PHQ8_Binary
0    77
1    30
Name: count, dtype: int64
The samples in back dataset is:  136 and the distribution is  PHQ8_Binary
0    77
1    59
Name: count, dtype: int64
The samples in aug dataset is:  127 and the distribution is  PHQ8_Binary
0    77
1    50
Name: count, dtype: int64
The samples in validation dataset is:  35 and the distribution is  PHQ8_Binary
0    23
1    12
Name: count, dtype: int64
The samples in test dataset is:  47 and the distribution is  PHQ8_Binary
0    33
1    14
Name: count, dtype: int64


Let's start with the training dataset. First of all we will do the classification without under/oversampling, using tf-idf, word2vec and glove. After which we will use sampling balancing. We will the test in the validation set and subsequently test set. We will do the same for back and aug dataset. At last we, will also try incorporating val into training dataset, as we have a separate test dataset for testing.

<h1>TF-IDF</h1>
<h3>Train_set</h3>

In [4]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6100)

X_train_tfidf=tfidf_vectorizer.fit_transform(train_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_train=train_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [67]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=2000.0,gamma='auto')
svm.fit(X_train_tfidf,y_train)
y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.61      0.65        23
  Depression       0.40      0.50      0.44        12

    accuracy                           0.57        35
   macro avg       0.55      0.55      0.55        35
weighted avg       0.60      0.57      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.81      0.79      0.80        33
  Depression       0.53      0.57      0.55        14

    accuracy                           0.72        47
   macro avg       0.67      0.68      0.68        47
weighted avg       0.73      0.72      0.73        47



In [139]:
#Random Undersampling
rus=RandomUnderSampler(random_state=7)
X_train_tfidf_un,y_train_un=rus.fit_resample(X_train_tfidf,y_train)
svm = SVC(kernel='rbf',random_state=42,class_weight='balanced',C=10000.00,gamma='auto')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_train_tfidf_un,y_train_un)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.75      0.65      0.70        23
  Depression       0.47      0.58      0.52        12

    accuracy                           0.63        35
   macro avg       0.61      0.62      0.61        35
weighted avg       0.65      0.63      0.64        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.86      0.73      0.79        33
  Depression       0.53      0.71      0.61        14

    accuracy                           0.72        47
   macro avg       0.69      0.72      0.70        47
weighted avg       0.76      0.72      0.73        47



In [162]:
#Random OverSampling
smote=SMOTE()
X_train_tfidf_smote,y_train_smote=smote.fit_resample(X_train_tfidf,y_train)
svm = SVC(kernel='sigmoid',random_state=7,class_weight='balanced',C=100.00,gamma='scale')
svm.fit(X_train_tfidf_smote,y_train_smote)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.87      0.77        23
  Depression       0.50      0.25      0.33        12

    accuracy                           0.66        35
   macro avg       0.59      0.56      0.55        35
weighted avg       0.62      0.66      0.62        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.75      0.82      0.78        33
  Depression       0.45      0.36      0.40        14

    accuracy                           0.68        47
   macro avg       0.60      0.59      0.59        47
weighted avg       0.66      0.68      0.67        47



<h3>Back_Set</h3>

In [164]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6100)

X_back_tfidf=tfidf_vectorizer.fit_transform(back_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_back=back_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [182]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=700.00,gamma='auto')
svm.fit(X_back_tfidf,y_back)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.39      0.50        23
  Depression       0.36      0.67      0.47        12

    accuracy                           0.49        35
   macro avg       0.53      0.53      0.49        35
weighted avg       0.58      0.49      0.49        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.81      0.52      0.63        33
  Depression       0.38      0.71      0.50        14

    accuracy                           0.57        47
   macro avg       0.60      0.61      0.56        47
weighted avg       0.68      0.57      0.59        47



In [200]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_back_tfidf_un,y_back_un=rus.fit_resample(X_back_tfidf,y_back)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=10000.00,gamma='scale')

svm.fit(X_back_tfidf_un,y_back_un)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.78      0.72        23
  Depression       0.38      0.25      0.30        12

    accuracy                           0.60        35
   macro avg       0.52      0.52      0.51        35
weighted avg       0.57      0.60      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.78      0.85      0.81        33
  Depression       0.55      0.43      0.48        14

    accuracy                           0.72        47
   macro avg       0.66      0.64      0.65        47
weighted avg       0.71      0.72      0.71        47



In [201]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_back_tfidf_smote,y_back_smote=smote.fit_resample(X_back_tfidf,y_back)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=10.00,gamma='scale')
svm.fit(X_back_tfidf_smote,y_back_smote)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.87      0.75        23
  Depression       0.40      0.17      0.24        12

    accuracy                           0.63        35
   macro avg       0.53      0.52      0.50        35
weighted avg       0.58      0.63      0.58        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.88      0.82        33
  Depression       0.56      0.36      0.43        14

    accuracy                           0.72        47
   macro avg       0.66      0.62      0.63        47
weighted avg       0.70      0.72      0.70        47



<h3>Aug_Set</h3>

In [202]:
tfidf_vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_features=6200)

X_aug_tfidf=tfidf_vectorizer.fit_transform(aug_dataset['response'])
X_val_tfidf=tfidf_vectorizer.transform(val_dataset['response'])
X_test_tfidf=tfidf_vectorizer.transform(test_dataset['response'])

y_aug=aug_dataset['PHQ8_Binary']
y_val=val_dataset['PHQ8_Binary']
y_test=test_dataset['PHQ8_Binary']

In [234]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=650.0,gamma='auto')
svm.fit(X_aug_tfidf,y_aug)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.39      0.50        23
  Depression       0.36      0.67      0.47        12

    accuracy                           0.49        35
   macro avg       0.53      0.53      0.49        35
weighted avg       0.58      0.49      0.49        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.84      0.48      0.62        33
  Depression       0.39      0.79      0.52        14

    accuracy                           0.57        47
   macro avg       0.62      0.64      0.57        47
weighted avg       0.71      0.57      0.59        47



In [263]:
#Random Undersampling
rus=RandomUnderSampler(random_state=7)
X_aug_tfidf_un,y_aug_un=rus.fit_resample(X_aug_tfidf,y_aug)
svm = SVC(kernel='linear',random_state=42,class_weight='balanced',C=1000.00,gamma='scale')

svm.fit(X_aug_tfidf_un,y_aug_un)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.91      0.79        23
  Depression       0.60      0.25      0.35        12

    accuracy                           0.69        35
   macro avg       0.65      0.58      0.57        35
weighted avg       0.67      0.69      0.64        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.74      0.88      0.81        33
  Depression       0.50      0.29      0.36        14

    accuracy                           0.70        47
   macro avg       0.62      0.58      0.58        47
weighted avg       0.67      0.70      0.67        47



In [289]:
#Random OverSampling
smote=SMOTE(random_state=77)
X_aug_tfidf_smote,y_aug_smote=smote.fit_resample(X_aug_tfidf,y_aug)
svm = SVC(kernel='sigmoid',random_state=7,class_weight='balanced',C=10.00,gamma='scale')
svm.fit(X_aug_tfidf_smote,y_aug_smote)

y_val_pred=svm.predict(X_val_tfidf)
y_test_pred=svm.predict(X_test_tfidf)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.69      0.87      0.77        23
  Depression       0.50      0.25      0.33        12

    accuracy                           0.66        35
   macro avg       0.59      0.56      0.55        35
weighted avg       0.62      0.66      0.62        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.85      0.80        33
  Depression       0.50      0.36      0.42        14

    accuracy                           0.70        47
   macro avg       0.63      0.60      0.61        47
weighted avg       0.68      0.70      0.69        47



<h1>Word2Vec</h1>
<h3>Train_set</h3>

In [290]:
import gensim

word2vec_path='..\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin'
word2vec=gensim.models.KeyedVectors.load_word2vec_format(word2vec_path,binary=True)

In [291]:
def get_average_word2vec(tokens_list,vector,k=300):
    valid_vectors=[vector[word] for word in tokens_list if word in vector]

    if not valid_vectors:
        print(tokens_list)
        return np.zeros(k)
        
    
    return np.mean(valid_vectors,axis=0)

In [292]:
train_dataset['tokens']=train_dataset['response'].apply(lambda x:x.split())
val_dataset['tokens']=val_dataset['response'].apply(lambda x:x.split())
test_dataset['tokens']=test_dataset['response'].apply(lambda x:x.split())
X_train_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in train_dataset['tokens'] ])
X_val_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in val_dataset['tokens']])
X_test_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in test_dataset['tokens']])

In [303]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=22500.0,gamma='auto')
svm.fit(X_train_word2vec,y_train)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.80      0.70      0.74        23
  Depression       0.53      0.67      0.59        12

    accuracy                           0.69        35
   macro avg       0.67      0.68      0.67        35
weighted avg       0.71      0.69      0.69        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.83      0.76      0.79        33
  Depression       0.53      0.64      0.58        14

    accuracy                           0.72        47
   macro avg       0.68      0.70      0.69        47
weighted avg       0.74      0.72      0.73        47



In [304]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_train_word2vec_un,y_train_word2vec_un=rus.fit_resample(X_train_word2vec,y_train)

In [313]:
rus=RandomUnderSampler(random_state=42)
X_train_word2vec_un,y_train_word2vec_un=rus.fit_resample(X_train_word2vec,y_train)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=32500.0,gamma='auto')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_train_word2vec_un,y_train_word2vec_un)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.73      0.70      0.71        23
  Depression       0.46      0.50      0.48        12

    accuracy                           0.63        35
   macro avg       0.59      0.60      0.60        35
weighted avg       0.64      0.63      0.63        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.85      0.67      0.75        33
  Depression       0.48      0.71      0.57        14

    accuracy                           0.68        47
   macro avg       0.66      0.69      0.66        47
weighted avg       0.74      0.68      0.69        47



In [316]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_train_word2vec_smote,y_train_word2vec_smote=smote.fit_resample(X_train_word2vec,y_train)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=22500.0,gamma='auto')
svm.fit(X_train_word2vec_smote,y_train_word2vec_smote)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.70      0.73        23
  Depression       0.50      0.58      0.54        12

    accuracy                           0.66        35
   macro avg       0.63      0.64      0.63        35
weighted avg       0.67      0.66      0.66        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.85      0.70      0.77        33
  Depression       0.50      0.71      0.59        14

    accuracy                           0.70        47
   macro avg       0.68      0.71      0.68        47
weighted avg       0.75      0.70      0.71        47



<h3>Back_set</h3>

In [317]:
back_dataset['tokens']=back_dataset['response'].apply(lambda x:x.split())
X_back_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in back_dataset['tokens'] ])

In [332]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=52500.0,gamma='auto')
svm.fit(X_back_word2vec,y_back)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.72      0.78      0.75        23
  Depression       0.50      0.42      0.45        12

    accuracy                           0.66        35
   macro avg       0.61      0.60      0.60        35
weighted avg       0.64      0.66      0.65        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.79      0.78        33
  Depression       0.46      0.43      0.44        14

    accuracy                           0.68        47
   macro avg       0.61      0.61      0.61        47
weighted avg       0.67      0.68      0.68        47



In [333]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_back_word2vec_un,y_back_word2vec_un=rus.fit_resample(X_back_word2vec,y_back)

In [342]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=12500.0,gamma='scale')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_back_word2vec_un,y_back_word2vec_un)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.80      0.70      0.74        23
  Depression       0.53      0.67      0.59        12

    accuracy                           0.69        35
   macro avg       0.67      0.68      0.67        35
weighted avg       0.71      0.69      0.69        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.61      0.66        33
  Depression       0.32      0.43      0.36        14

    accuracy                           0.55        47
   macro avg       0.52      0.52      0.51        47
weighted avg       0.60      0.55      0.57        47



In [355]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_back_word2vec_smote,y_back_word2vec_smote=smote.fit_resample(X_back_word2vec,y_back)
svm = SVC(kernel='linear',random_state=42,class_weight='balanced',C=500.0,gamma='scale')
svm.fit(X_back_word2vec_smote,y_back_word2vec_smote)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.83      0.75        23
  Depression       0.43      0.25      0.32        12

    accuracy                           0.63        35
   macro avg       0.55      0.54      0.53        35
weighted avg       0.59      0.63      0.60        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.74      0.76      0.75        33
  Depression       0.38      0.36      0.37        14

    accuracy                           0.64        47
   macro avg       0.56      0.56      0.56        47
weighted avg       0.63      0.64      0.63        47



<h3>Aug_set</h3>

In [356]:
aug_dataset['tokens']=aug_dataset['response'].apply(lambda x:x.split())
X_aug_word2vec=np.array([get_average_word2vec(tokens,word2vec) for tokens in aug_dataset['tokens'] ])

In [360]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=52500.0,gamma='auto')
svm.fit(X_aug_word2vec,y_aug)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.70      0.70        23
  Depression       0.42      0.42      0.42        12

    accuracy                           0.60        35
   macro avg       0.56      0.56      0.56        35
weighted avg       0.60      0.60      0.60        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.81      0.76      0.78        33
  Depression       0.50      0.57      0.53        14

    accuracy                           0.70        47
   macro avg       0.65      0.66      0.66        47
weighted avg       0.72      0.70      0.71        47



In [361]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_aug_word2vec_un,y_aug_word2vec_un=rus.fit_resample(X_aug_word2vec,y_aug)

In [368]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=42500.0,gamma='auto')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_aug_word2vec_un,y_aug_word2vec_un)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.70      0.73        23
  Depression       0.50      0.58      0.54        12

    accuracy                           0.66        35
   macro avg       0.63      0.64      0.63        35
weighted avg       0.67      0.66      0.66        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.86      0.76      0.81        33
  Depression       0.56      0.71      0.63        14

    accuracy                           0.74        47
   macro avg       0.71      0.74      0.72        47
weighted avg       0.77      0.74      0.75        47



In [378]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_aug_word2vec_smote,y_aug_word2vec_smote=smote.fit_resample(X_aug_word2vec,y_aug)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=12500.0,gamma='auto')
svm.fit(X_aug_word2vec_smote,y_aug_word2vec_smote)

y_val_pred=svm.predict(X_val_word2vec)
y_test_pred=svm.predict(X_test_word2vec)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.72      0.78      0.75        23
  Depression       0.50      0.42      0.45        12

    accuracy                           0.66        35
   macro avg       0.61      0.60      0.60        35
weighted avg       0.64      0.66      0.65        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.76      0.88      0.82        33
  Depression       0.56      0.36      0.43        14

    accuracy                           0.72        47
   macro avg       0.66      0.62      0.63        47
weighted avg       0.70      0.72      0.70        47



<h1>GlOvE</h1>
<h3>Train_Set</h3>

In [379]:
embedding_index={}
glove_path='../glove.6B.100d.txt'
with open(glove_path,'r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        coefs=np.asarray(values[1:],dtype='float32')
        embedding_index[word]=coefs

In [380]:
def get_average_glove(tokens_list,embedding_index,k=100):
    valid_vectors=[embedding_index[word] for word in tokens_list if word in embedding_index]

    if not valid_vectors:
        return np.zeros(k)
    
    return np.mean(valid_vectors,axis=0)

In [381]:
X_train_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in train_dataset['tokens']])
X_back_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in back_dataset['tokens']])
X_aug_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in aug_dataset['tokens']])
X_val_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in val_dataset['tokens']])
X_test_glove=np.array([get_average_glove(tokens,embedding_index) for tokens in test_dataset['tokens']])

In [420]:
svm = SVC(kernel='rbf',random_state=42,class_weight='balanced',C=90000.00,gamma='auto')
svm.fit(X_train_glove,y_train)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.74      0.71        23
  Depression       0.40      0.33      0.36        12

    accuracy                           0.60        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.58      0.60      0.59        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.80      0.73      0.76        33
  Depression       0.47      0.57      0.52        14

    accuracy                           0.68        47
   macro avg       0.64      0.65      0.64        47
weighted avg       0.70      0.68      0.69        47



In [421]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_train_glove_un,y_train_glove_un=rus.fit_resample(X_train_glove,y_train)

In [430]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=500.0,gamma='scale')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_train_glove_un,y_train_glove_un)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.74      0.72        23
  Depression       0.45      0.42      0.43        12

    accuracy                           0.63        35
   macro avg       0.58      0.58      0.58        35
weighted avg       0.62      0.63      0.62        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.84      0.64      0.72        33
  Depression       0.45      0.71      0.56        14

    accuracy                           0.66        47
   macro avg       0.65      0.68      0.64        47
weighted avg       0.73      0.66      0.67        47



In [431]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_train_glove_smote,y_train_glove_smote=smote.fit_resample(X_train_glove,y_train)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=22500.0,gamma='auto')
svm.fit(X_train_glove_smote,y_train_glove_smote)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.74      0.71        23
  Depression       0.40      0.33      0.36        12

    accuracy                           0.60        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.58      0.60      0.59        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.84      0.82      0.83        33
  Depression       0.60      0.64      0.62        14

    accuracy                           0.77        47
   macro avg       0.72      0.73      0.73        47
weighted avg       0.77      0.77      0.77        47



<h3>Back_set</h3>

In [436]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=22500.0,gamma='auto')
svm.fit(X_back_glove,y_back)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.70      0.83      0.76        23
  Depression       0.50      0.33      0.40        12

    accuracy                           0.66        35
   macro avg       0.60      0.58      0.58        35
weighted avg       0.63      0.66      0.64        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.83      0.88      0.85        33
  Depression       0.67      0.57      0.62        14

    accuracy                           0.79        47
   macro avg       0.75      0.73      0.73        47
weighted avg       0.78      0.79      0.78        47



In [437]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_back_glove_un,y_back_glove_un=rus.fit_resample(X_back_glove,y_back)

In [439]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=12500.0,gamma='auto')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_back_glove_un,y_back_glove_un)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.65      0.65      0.65        23
  Depression       0.33      0.33      0.33        12

    accuracy                           0.54        35
   macro avg       0.49      0.49      0.49        35
weighted avg       0.54      0.54      0.54        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.83      0.88      0.85        33
  Depression       0.67      0.57      0.62        14

    accuracy                           0.79        47
   macro avg       0.75      0.73      0.73        47
weighted avg       0.78      0.79      0.78        47



In [450]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_back_glove_smote,y_back_glove_smote=smote.fit_resample(X_back_glove,y_back)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=62500.0,gamma='auto')
svm.fit(X_back_glove_smote,y_back_glove_smote)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.68      0.74      0.71        23
  Depression       0.40      0.33      0.36        12

    accuracy                           0.60        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.58      0.60      0.59        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.82      0.82      0.82        33
  Depression       0.57      0.57      0.57        14

    accuracy                           0.74        47
   macro avg       0.69      0.69      0.69        47
weighted avg       0.74      0.74      0.74        47



<h3>Aug_set</h3>

In [455]:
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=32500.0,gamma='auto')
svm.fit(X_aug_glove,y_aug)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.74      0.72        23
  Depression       0.45      0.42      0.43        12

    accuracy                           0.63        35
   macro avg       0.58      0.58      0.58        35
weighted avg       0.62      0.63      0.62        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.79      0.82      0.81        33
  Depression       0.54      0.50      0.52        14

    accuracy                           0.72        47
   macro avg       0.67      0.66      0.66        47
weighted avg       0.72      0.72      0.72        47



In [456]:
#Random Undersampling
rus=RandomUnderSampler(random_state=42)
X_aug_glove_un,y_aug_glove_un=rus.fit_resample(X_aug_glove,y_aug)

In [463]:
svm = SVC(kernel='linear',random_state=42,class_weight='balanced',C=412500.0,gamma='scale')
# svm=LogisticRegression(max_iter=1000)
svm.fit(X_aug_glove_un,y_aug_glove_un)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.67      0.61      0.64        23
  Depression       0.36      0.42      0.38        12

    accuracy                           0.54        35
   macro avg       0.51      0.51      0.51        35
weighted avg       0.56      0.54      0.55        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.78      0.64      0.70        33
  Depression       0.40      0.57      0.47        14

    accuracy                           0.62        47
   macro avg       0.59      0.60      0.59        47
weighted avg       0.67      0.62      0.63        47



In [464]:
#Random OverSampling
smote=SMOTE(random_state=42)
X_aug_glove_smote,y_aug_glove_smote=smote.fit_resample(X_aug_glove,y_aug)
svm = SVC(kernel='sigmoid',random_state=42,class_weight='balanced',C=22500.0,gamma='auto')
svm.fit(X_aug_glove_smote,y_aug_glove_smote)

y_val_pred=svm.predict(X_val_glove)
y_test_pred=svm.predict(X_test_glove)

print('Validation Set Performance:')
print(classification_report(y_val,y_val_pred,target_names=['Controlled','Depression'],zero_division=0.0))

print('Test Set Performance:')
print(classification_report(y_test,y_test_pred,target_names=['Controlled','Depression'],zero_division=0.0))

Validation Set Performance:
              precision    recall  f1-score   support

  Controlled       0.71      0.74      0.72        23
  Depression       0.45      0.42      0.43        12

    accuracy                           0.63        35
   macro avg       0.58      0.58      0.58        35
weighted avg       0.62      0.63      0.62        35

Test Set Performance:
              precision    recall  f1-score   support

  Controlled       0.80      0.85      0.82        33
  Depression       0.58      0.50      0.54        14

    accuracy                           0.74        47
   macro avg       0.69      0.67      0.68        47
weighted avg       0.74      0.74      0.74        47

